# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [73]:

## make sure the following dependencies are installed, specifically geopy
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import requests
import scipy.stats
import random as rd
from citipy import citipy
import csv
import os

#to import keys without having them in my repo
import sys
sys.path.insert(1,'/Users/Trav/Desktop/Coding/GitHub_Repositories/API_PACKAGE')
import api_keys



## Generate Cities List

In [75]:

## generates 2 lists of random floats for latitude and longitude
### maybe I could generate a larger list and narrow it down by population size(whats my max, 5000????)
samp_size = 1500
lat_list = [round(rd.uniform(-90,90),2) for num in range(samp_size)]
lon_list = [round(rd.uniform(-180,180),2) for num in range(samp_size)]

city_name_list=[]
country_code_list=[]
ex_coor_list=[]

#turns the keys and values from the world cities dict into lists
new_world_keys=list(citipy.WORLD_CITIES_DICT.keys())
new_world_values=list(citipy.WORLD_CITIES_DICT.values())

# generating data in a for loop
for y in range(samp_size):
    #grabs the approximate latitude and longitutde
    ap_lat=lat_list[y]
    ap_lon=lon_list[y]
    #use citipy functions to get the city name and country code
    current_city=citipy.nearest_city(ap_lat,ap_lon)
    city_name_list.append(current_city.city_name)
    country_code_list.append(current_city.country_code)
    # uses the new world lists to get the exact coordinates for the city
    nwvi=new_world_values.index(current_city)
    ex_coor_list.append(new_world_keys[nwvi])
 
# turns the lists created in the for loop into a dataframe
## i might want to remove the 2 for list comprehensions below and do it in the for loop above
coor_df= pd.DataFrame({
    'Approximate Lat':lat_list,
    'Approximate Lon':lon_list,
    'City Name':city_name_list,
    'Country Code': country_code_list,
    'Exact Coordinates':ex_coor_list,
    'Latitude':[i[0] for i in ex_coor_list],
    'Longitude':[i[1] for i in ex_coor_list],
})

#removes dupliactes based on the exact coordinates
coor_df=coor_df.drop_duplicates(subset=['Exact Coordinates'],keep='first')

#trims down the dataframe to 500 if there are more 
coor_df_len = len(coor_df['Approximate Lat'])
if coor_df_len > 500:
    coor_df.drop(coor_df.tail(coor_df_len-500).index,inplace=True)

# coor_df.head()


### Perform API Calls and turn raw data into dataframe
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).
* Display the DataFrame

In [77]:

##sets up the query url for the lat/lon weather request #need to look into git ignore
base_url = "http://api.openweathermap.org/data/2.5/weather?"

coor_df['Temperature']=''
coor_df['Humidity']=''
coor_df['Cloudiness']=''
coor_df['Wind Speed']=''

for i,r in coor_df.iterrows():
    #gets the lat and lon from the df
    wea_coors = coor_df["Exact Coordinates"][i]
    wea_lat=wea_coors[0]
    wea_lon=wea_coors[1]
    #send api to requestthe temperature, humidity, cloudiness, and wind speed 
    query_url =f'{base_url}lat={wea_lat}&lon={wea_lon}&appid={api_keys.weather_api_key}&units=imperial'
    wea_json = requests.get(query_url).json()
    coor_df['Temperature'][i]=wea_json["main"]['temp']
    coor_df['Humidity'][i]=wea_json['main']['humidity']
    coor_df['Cloudiness'][i]=wea_json['clouds']['all']
    coor_df['Wind Speed'][i]=wea_json['wind']['speed']

coor_df.head(20)



,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,54.98,-117.91,valleyview,ca,"(55.066667, -117.283333)",55.066667,-117.283333,30.88,95,99,7.36
1,53.60,-13.37,dingle,ie,"(52.1408333, -10.2688889)",52.140833,-10.268889,52.36,83,100,10.71
2,-32.08,-16.69,jamestown,sh,"(-15.9333333, -5.7166667)",-15.933333,-5.716667,62.64,89,100,23.49
3,37.95,99.76,zhangye,cn,"(38.934167, 100.451667)",38.934167,100.451667,46.98,48,4,16.04
4,49.51,140.54,vanino,ru,"(49.086856, 140.254337)",49.086856,140.254337,44.83,64,24,6.89
5,43.27,66.15,shieli,kz,"(44.166667, 66.75)",44.166667,66.750000,48.38,46,0,14.61
6,-79.34,-140.51,rikitea,pf,"(-23.1202778, -134.9691667)",-23.120278,-134.969167,75.49,74,90,5.26
7,-84.85,107.73,albany,au,"(-35.003101, 117.865952)",-35.003101,117.865952,77,33,0,12.75
8,-17.19,81.65,hithadhoo,mv,"(-0.6, 73.0833333)",-0.600000,73.083333,83.1,77,60,13
9,-54.17,52.88,taolanaro,mg,"(-25.0333333, 47.0)",-25.033333,47.000000,71.6,78,40,16.11


### Convert Dataframe to csv
* Export the city data into a .csv.


In [79]:
# #create a new folder and exports the dataframe

def dataframe_to_csv(df):
    current_directory=os.getcwd()
    new_folder_path =os.path.join(current_directory,r'Export Folder')
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
    csv_export_path = os.path.join(new_folder_path,'city_data.csv')
    df.to_csv(csv_export_path,index=False)
dataframe_to_csv(coor_df)

In [81]:
## import the csv file you just saved
def import_path():
    current_directory=os.getcwd()
    new_folder_path =os.path.join(current_directory,r'Export Folder')
    csv_export_path = os.path.join(new_folder_path,'city_data.csv')
    return csv_export_path

im_path=import_path()
print(im_path)
csv_imp_df=pd.read_csv(im_path,delimiter=',')
csv_imp_df.head()

c:\Users\Trav\Desktop\Coding\GitHub_Repositories\Weather_API\Weather\Export Folder\city_data.csv


,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,54.98,-117.91,valleyview,ca,"(55.066667, -117.283333)",55.066667,-117.283333,30.88,95,99,7.36
1,53.60,-13.37,dingle,ie,"(52.1408333, -10.2688889)",52.140833,-10.268889,52.36,83,100,10.71
2,-32.08,-16.69,jamestown,sh,"(-15.9333333, -5.7166667)",-15.933333,-5.716667,62.64,89,100,23.49
3,37.95,99.76,zhangye,cn,"(38.934167, 100.451667)",38.934167,100.451667,46.98,48,4,16.04
4,49.51,140.54,vanino,ru,"(49.086856, 140.254337)",49.086856,140.254337,44.83,64,24,6.89


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [82]:
## creates a new dataframe, excluding humidity over 100
####### i don't like where this is placed considering total cities could be less than 500, i think I should drop the high humidity rows when im iterating through the dataframe(or after i iterate through it), that way my initial csv export is clean as well.
print(len(csv_imp_df['Humidity']))
city_weather_df=csv_imp_df.loc[csv_imp_df['Humidity']<=100,:]
print(len(city_weather_df['Humidity']))

500
500


In [83]:
### set up the plot
x_limit=round(city_weather_df['Temperature'].max(),-1)

# y_limit=round(city_weather_df['Exact Coordinates'][0].max(),-1)



## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression